#  **AULA 3 P1: Censo Americano**

In [1]:
# importa as bibliotecas
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

Primeiro, é necessário carregar o dataset. Para baixar o dataset e mais informações, acesse: http://archive.ics.uci.edu/ml/datasets/Adult

Este dataset contém informações do Censo Americano e o objetivo dele é predizer se uma determinada pessoa tem ganho maior que 50K em um ano. Essa predição é baseada nas informações sociais coletadas no Censo. Essa situação poderia ser utilizada para obter um empréstimo ou não (se ganha mais de 50K libera o empréstimo). 

In [ ]:
# importa o data set
from google.colab import files
uploaded = files.upload()

# para baixar o dataset, acesse: 
# Observação 1: Este dataset já é dividido em treino e teste previamente. Porém, vou usar somente o conjunto de 
# teste como sendo o dataset original, pois a maioria dos datasets não são previamentes divididos em conjuntos
# de teste e conjunto de treino (isso é feito no processo de Aprendizado de Máquina)

# do site eu recuperei o nome de cada coluna do data set
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
           'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 
           'target']

# abro o dataset e nomeio cada coluna com meu vetor columns
data = pd.read_csv('adult.data', names=columns)

# Fazendo a mesma coisa do Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
# data = pd.read_csv('/content/drive/My Drive/EABDA 2020/adult.data', names=columns)

# elimina os espaços no começo e no fim das strings
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# visualiza a tabela, o método head() mostra as 5 primeiras linhas da tabela
# note que a coluna 'target' é a variável dependente ou resposta (classe).
# o restante das colunas são as variáveis explicativas ou independentes ou preditoras (features, atributos), e cada linha é um indivíduo (instância ou exemplo).
data.head()

In [ ]:
# mostra o número de linhas (unidades amostrais) e colunas (variáveis)
data.shape

Geralmente, os datasets vem com dados faltantes, no caso deste dataset os dados faltantes é simbolizado pelo caractere '?'. Existem diversas estratégias para trabalhar com dados faltantes. Algumas metodologias podem lidar com "missings" sem nenhum problema, mas outras necessitam de dados completos. Nesses casos, pode-se imputar dados. Há formas sofisticadas e simples de imputação, sendo uma delas a substituição pelo valor médio. Porém, no nosso caso, por questões didáticas e de simplificação, vou excluir todos os dados faltantes da nossa base de dados. 

In [ ]:
# elimina as linhas com dados faltantes
# data = pd.DataFrame
# null_symbol = caractere que simboliza que há dados faltantes
def remove_rows(data, null_symbol=np.nan):
    # salva o total de exemplos
    total = len(data)
    # substitui o null_symbol por nulo
    data = data.replace(null_symbol, np.nan)
    # elimina as linhas que contém algum valor nulo
    data.dropna(inplace=True)
    print('Execute: preprocessing_remove_rows\n---Removed %d rows in the table' % (total - len(data)))
    return data

data = remove_rows(data, '?')

In [ ]:
# o tamanho atualizado do conjunto de dados
data.shape

Teste para a comparação de 2 proporções: amostras independentes


In [ ]:
# Há injustiça no valor salarial?
c = data["target"].value_counts(sort=False)
p = data["target"].value_counts(normalize=True,sort=False).round(3) * 100
pd.concat([c,p], axis=1, keys=['counts', '%'])


In [ ]:
# Tabela de frequências cruzadas ou tabela de contingência
pd.crosstab(data.sex, data.target,margins=True)

In [ ]:
# Percentual de linhas
pd.crosstab(data.sex, data.target,normalize='index')

In [ ]:
from scipy.stats import chi2_contingency

# teste quiquadrado para independência/hamogeneidade de distribuição
obs = pd.crosstab(data.sex, data.target)
chi2_contingency(obs)

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

count = np.array([1112, 6396])
nobs = np.array([9782, 20380])
stat, pval = proportions_ztest(count, nobs)
print(stat,pval)

In [ ]:
from scipy.stats import norm

# Qual o valor de Z ~ N(0,1) que deixa 5% dos dados à esquerda : F(z)=0,05
z= norm.ppf(0.05)     
print(z)


In [ ]:
# Estatística do z teste
estat = (0.113678-0.313837)/np.sqrt(0.249*(1-0.249)*(1/9782+1/20380))
estat

In [ ]:
# valor p
#2*norm.cdf(estat) # para o teste bicaudal, tem-se que mutiplicar por 2
norm.cdf(estat)

No R, o mesmo teste é feito com:

count = c(1112, 6396)

nobs = c(9782, 20380)

prop.test(count, nobs, p = NULL, alternative = "two.sided", correct = FALSE)



Testes t de Student

In [ ]:
dataF = data.loc[data.sex=='Female', "age"]
dataM = data.loc[data.sex=='Male', "age"]

data.groupby('sex').agg({'age': ['count',min, max, 'median','mean', 'std']})

In [ ]:
from scipy import stats

# Teste t de Student (bicaudal) para média de duas populações Normais com variâncias iguais
stats.ttest_ind(dataF,dataM, equal_var = True)

In [ ]:
import math
(36.883459-39.184004)/math.sqrt((9782*13.532427**2+20380*12.873243**2)/(9782+20380-2)*(1/9782+1/20380))

In [ ]:
# Teste de Levene para igualdade de variâncias
# stats.levene(dataF,dataM)

# Teste de Bartlett para igualdade de variâncias
stats.bartlett(dataF,dataM)


In [ ]:
# Selecione uma amostra de 95% e outra de 0,5% e repare como o resultado do teste muda
sampleF=dataF.sample(frac=0.005)
sampleM=dataM.sample(frac=0.005)

# Teste t de Student (bicaudal) para média de duas populações Normais com variâncias iguais
stats.ttest_ind(sampleF,sampleM, equal_var = True)

In [ ]:
sampleF.describe()

In [ ]:
sampleM.describe()

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Gráfico de probabilidade da Normal, para checar suposição de tal distribuição dos dados
# para o teste t de Student
stats.probplot(sampleF, plot=plt)
stats.probplot(sampleM, plot=plt)
fig = plt.figure()

In [ ]:
# Teste de Shapiro-Wilks para normalidade
stats.shapiro(dataM)

In [ ]:
# Teste de Kolmogorov-Smirnov para normalidade
mu = dataF.mean()
sigma = dataF.std()
stats.kstest(dataF,'norm',args=(mu,sigma))

In [ ]:
# Teste de Levene para igualdade de variâncias
stats.levene(sampleF,sampleM)


#  **AULA 3 P1: Projeto 2 (Tempo de realização de tarefas: descritiva, regressão e classificação)**

In [ ]:
# importa o data set
from google.colab import files
uploaded = files.upload()

data2 = pd.read_csv('projeto2.csv')
data2.head()

In [ ]:
data2.head()

In [ ]:
# o tamanho atualizado do conjunto de dados
data2.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
 
# Grouped boxplot
box_plot_data=[data2.tA,data2.tB,data2.tC]
plt.boxplot(box_plot_data,patch_artist=True,labels=['Descritiva','Regressão','Classificação'])
plt.show()


In [ ]:
data2.describe().round(decimals=2)

In [ ]:
import scipy.stats as stats

# ANOVA usando scipy.stats.f_oneway: cada grupo em uma coluna
stats.f_oneway(data2.tA, data2.tB, data2.tC)

In [ ]:
# Teste de homocedasticidade
stats.levene(data2.tA, data2.tB, data2.tC)

In [ ]:

from google.colab import files
uploaded = files.upload()

data3 = pd.read_csv('projeto2_colunas.csv')
data3.head()

In [ ]:
# Tabela ANOVA (como em R)
import statsmodels.api as sm
from statsmodels.formula.api import ols

# os tempo devem estar em uma única coluna
model = ols('tempo ~ C(tarefa)',data=data3).fit()
anova_table = sm.stats.anova_lm(model, typ=2) # ou typ=3, para mais fatores
anova_table

In [ ]:
# valores preditos da resposta média
ypred=model.fittedvalues

# Calcula os resíduos (observado-ajustado) do modelo para verificação das suposições
res = model.resid

# objeto para a análise de pontos influentes
infl = model.get_influence()

# resíduo studentizado (internamente)
res_stud = infl.resid_studentized_internal

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
ax1.scatter(ypred, res)
ax1.set_ylabel('$y-\hat{y}$')
ax1.set_title('Resíduos')
ax1.hlines(0,xmin=min(ypred),xmax=max(ypred),color='gray')
ax2.scatter(ypred, res_stud)
ax2.set_ylabel('studentizado')
ax2.set_xlabel('valor predito')
ax2.hlines(0,xmin=min(ypred),xmax=max(ypred),color='gray')

for ax in fig.get_axes():
    ax.label_outer()

In [ ]:
# Teste de normalidade dos resíduos
stats.shapiro(res)

In [ ]:
# Gráfico de probabilidade Normal
fig = sm.qqplot(res, line='s')

In [ ]:
# Comparações múltiplas

from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)

# Método de comparações múltiplas de Tukey
MultiComp = MultiComparison(data3.tempo,
                           data3.tarefa)
print(MultiComp.tukeyhsd().summary())

In [ ]:
# Holm-Bonferroni method 
comp = MultiComp.allpairtest(stats.ttest_rel, method='Holm')
print (comp[0])


In [ ]:
# Vamos supor que as bases de dados são as mesmas e comparar se os tempos das tarefas de descrição e regressão 
import matplotlib.pyplot as plt

data2.dif = data2.tA-data2.tB
plt.boxplot(data2.dif,labels=['tA-tB'])

In [ ]:
from statistics import mean, stdev
media = mean(data2.dif)
dp = stdev(data2.dif)
est = media/dp*np.sqrt(50)

media, dp,est

In [ ]:
from scipy.stats import t

t.ppf(0.025, 49)

In [ ]:
from scipy import stats
# Teste t de Student pareado
stats.ttest_rel(data2.tA, data2.tB)

In [ ]:
# Ou pelo teste para 1 população (equivalente à diferença dos pares de valores igual a 0)
stats.ttest_1samp(data2.tA-data2.tB, 0)